# Assignment 1: This is the title of the assignment

Authors:  
Shayan Paydar Darian and 
Daniel Pazirai 


## Imports

In [ ]:
# It is recommended to start with general import statements
from utility_functions import *

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [ ]:
# Load the Swedish Medical NER dataset

# Check the dataset structure
# print(dataset)

## Task 1: Survey (kolla upp vilka modeller som finns och kolla vilken LLN som är bäst för Swedish Biomedical NER)

## Survey of Models for Swedish Biomedical NER

| **Model**        | **Language(s)** | **Strengths**                                   | **Weaknesses**                                  | **Relevance for Biomedical NER (Swedish)**               |
|------------------|-----------------|-------------------------------------------------|-------------------------------------------------|----------------------------------------------------------|
| **KB-BERT**      | Swedish         | Native Swedish, decent at general NER tasks     | No biomedical pretraining                       | Good starting point                                      |
| **Swedish ALBERT** | Swedish       | Lightweight, faster to train                    | Lacks domain specialization                    | OK, but may miss nuances                                 |
| **mBERT**        | Multilingual    | Cross-lingual abilities                         | Not optimized for any specific language/domain | Medium                                                   |
| **XLM-RoBERTa**  | Multilingual    | Robust multilingual understanding               | Same as mBERT, no biomedical specialization    | Medium                                                   |
| **BioBERT (English)** | English    | Biomedical domain specialized                   | Needs translated data, loses Swedish nuances   | Very good if translation is perfect                      |
 
 The survey results are based on searches on HuggingFace and google scholar with keywords (Swedish NER, Biomedical NER, etc), in order to see what models there are and what models have been used before.


## Approach and Motivation
In our approach, we want to use a Swedish Language-Model rather than translating the existing data into English. Although models like BioBERT offer very good performance in biomedical NER tasks, translation of clinical text can introduce errors and result in a loss of contextual information. This is particularly risky in the biomedical domain, where subtle differences in terminology can significantly impact the meaning of text. Additionally, as we are not biomedical experts, manually validating translations would be challenging.

Therefore, our approach is to fine-tune an existing Swedish model for biomedical NER, using the Swedish Medical NER dataset provided. 

The three models found on HuggingFace which were relevant were: 

bert-base-swedish-cased (v1) - A BERT trained with the same hyperparameters as first published by Google.

bert-base-swedish-cased-ner (experimental) - a BERT fine-tuned for NER using SUC 3.0.

albert-base-swedish-cased-alpha (alpha) - A first attempt at an ALBERT for Swedish.

## Our Choice
Out of of these models, we will be using the base model (bert-base-swedish-cased (v1)). Although the experimental version is fine tuned for NER, we want to train it on specifically biomedical NER. The base model will be perfect for that since it's flexible and gives us more control of fine tuning it and has less prior bias. We will also not use the Albert version since it's a first attempt at it and still in alpha.

Link for the KB bert models: https://huggingface.co/KB/bert-base-swedish-cased-ner 

Study which used KB/bert-base-swedish-cased and
bert-base-multilingual-cased: (Although not related to biomedicine) https://aclanthology.org/2024.caldpseudo-1.7.pdf 

It is mandatory to maintain the headings for each task.  
OPTIONALLY, you can use one level down (###) to organize subsessions of the assignments.

Use markdown cells like this one to include:
- Discussion points.
- References to specific sources of code that you might have used to solve the assignment.
- General commentas and explanations about your solution.

In [ ]:
# Always use comments in the code to document specific steps

## Task 2: Välj modell, ladda ner dataset som står (https://huggingface.co/datasets/community-datasets/swedish_medical_ner)

This section should contain the solution of task 2.

## Task 3: 

Preprocess the Dataset for Token Classification
Most likely, the data will be in token-label format. HuggingFace NER models expect:

Input IDs
Attention Masks
Labels per token
I can walk you through the actual code, but at a high level:

Tokenize the sentences.
Align the labels with the tokens.
Format into a HuggingFace dataset.

## Task 4:  Load the Model and Tokenizer och Fine-Tunea modellen

## Task 5: Evaluera och kolla var den failade, vilka resultat hade den? etc

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps